In [1]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt
import datetime
from dateutil.relativedelta import relativedelta
from pandas import DataFrame
from matplotlib.dates import drange
from sqlalchemy import func, and_, create_engine
from sqlalchemy import desc
from sqlalchemy import inspect
from dotenv import load_dotenv
import os

In [2]:
import numpy as np
import pandas as pd
import datetime as dt

In [3]:
#Read csv
csv_file = "Resources/Suburb_house.csv"
suburb_data_df = pd.read_csv(csv_file)
suburb_data_df.head()

ADDRESS      SUBURB   PRICE  BEDROOMS  BATHROOMS  GARAGE  \
0    1 Acorn Place  South Lake  565000         4          2     2.0   
1      1 Addis Way       Wandi  365000         3          2     2.0   
2  1 Ainsley Court     Camillo  287000         3          1     1.0   
3  1 Albert Street    Bellevue  255000         2          1     2.0   
4     1 Aman Place   Lockridge  325000         4          1     2.0   

   LAND_AREA  FLOOR_AREA  BUILD_YEAR  CBD_DIST               NEAREST_STN  \
0        600         160      2003.0     18300  Cockburn Central Station   
1        351         139      2013.0     26900           Kwinana Station   
2        719          86      1979.0     22600           Challis Station   
3        651          59      1953.0     17900           Midland Station   
4        466         131      1998.0     11200        Bassendean Station   

   NEAREST_STN_DIST  DATE_SOLD  POSTCODE   LATITUDE   LONGITUDE  \
0              1800  09-2018\r      6164 -32.115900  115.842450   
1              4900  02-2019\r      6167 -32.193470  115.859553   
2              1900  06-2015\r      6111 -32.120578  115.993579   
3              3600  07-2018\r      6056 -31.900547  116.038009   
4              2000  11-2016\r      6054 -31.885790  115.947780   

                    NEAREST_SCH  NEAREST_SCH_DIST  NEAREST_SCH_RANK  
0   LAKELAND SENIOR HIGH SCHOOL          0.828339               NaN  
1                ATWELL COLLEGE          5.524324             129.0  
2  KELMSCOTT SENIOR HIGH SCHOOL          1.649178             113.0  
3  SWAN VIEW SENIOR HIGH SCHOOL          1.571401               NaN  
4                 KIARA COLLEGE          1.514922               NaN

In [4]:
#Read csv
median_income = pd.read_csv('Resources/G02_median_incomes.csv')
median_income.head()

POA_CODE_2021  Median_age_persons  Median_mortgage_repay_monthly  \
0       POA6182                   0                              0   
1       POA6067                  33                              0   
2       POA6770                  29                           1800   
3       POA6718                  34                           1500   
4       POA6765                  31                           1625   

   Median_tot_prsnl_inc_weekly  Median_rent_weekly  Median_tot_fam_inc_weekly  \
0                            0                   0                          0   
1                          341                 150                        850   
2                          367                  90                        794   
3                          368                 150                       1006   
4                          386                 100                        863   

   Average_num_psns_per_bedroom  Median_tot_hhd_inc_weekly  \
0                           0.0                          0   
1                           1.9                        849   
2                           1.2                       1066   
3                           1.0                       1074   
4                           1.1                       1226   

   Average_household_size  
0                     0.0  
1                     3.0  
2                     3.2  
3                     2.9  
4                     3.0

In [5]:
#Read csv
csv_path = "Resources/solar_data_by_postcode.csv"

# Import the comic_books_expanded.csv file as a DataFrame
solar_df = pd.read_csv(csv_path, encoding="utf-8")
# Check the special characters imported correctly
print(solar_df.shape)
solar_df.head()

(2805, 45)


Small Unit Installation Postcode  \
0                              3029   
1                              4670   
2                              4740   
3                              4655   
4                              4350   

   Previous Years (2001-2020) - Installation Quantity  \
0                                              13028    
1                                              15863    
2                                              10650    
3                                              14573    
4                                              12326    

   Previous Years (2001-2020) - SGU Output Rated Output in kW  \
0                                          66465.309            
1                                          72687.831            
2                                          61880.971            
3                                          62724.539            
4                                          61933.825            

   Jan 2021 - Installations Quantity  Jan 2021 - SGU Rated Output In kW  \
0                                256                           1839.165   
1                                 93                            736.475   
2                                 85                            696.975   
3                                 91                            686.790   
4                                118                            863.025   

   Feb 2021 - Installations Quantity  Feb 2021 - SGU Rated Output In kW  \
0                                219                           1479.605   
1                                114                           1101.585   
2                                111                           1046.835   
3                                128                           1014.805   
4                                127                            991.905   

   Mar 2021 - Installations Quantity  Mar 2021 - SGU Rated Output In kW  \
0                                332                           2818.100   
1                                163                           1207.370   
2                                134                           1265.560   
3                                154                           1096.095   
4                                122                            998.090   

   Apr 2021 - Installations Quantity  ...  May 2022 - Installations Quantity  \
0                                275  ...                                112   
1                                152  ...                                110   
2                                 97  ...                                 95   
3                                144  ...                                100   
4                                128  ...                                 69   

   May 2022 - SGU Rated Output in kW  June 2022 - Installations Quantity  \
0                                965                                 136   
1                                784                                 125   
2                                775                                 114   
3                                707                                 105   
4                                681                                  99   

   June 2022 - SGU Rated Output in kW  July 2022 - Installations Quantity  \
0                            1080.890                                 151   
1                             946.100                                 106   
2                            1111.945                                  96   
3                             727.355                                 127   
4                             884.020                                  88   

   July 2022 - SGU Rated Output in kW  August 2022 - Installations Quantity  \
0                            1139.205                                    76   
1                             899.985                                   100   
2                             827.535      

In [6]:
load_dotenv()
protocol = 'postgresql'
username = os.environ.get('db_UserName')
password = os.environ.get('db_Password')
host = 'localhost'
port = 5432
database_name = 'Project_2'
rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(rds_connection_string)
insp = inspect(engine)


In [7]:
#Part 1
house_prices_df = suburb_data_df[["SUBURB","PRICE","BEDROOMS",	"BATHROOMS",	"GARAGE",	"LAND_AREA",	"FLOOR_AREA",	"CBD_DIST",	"NEAREST_STN_DIST",	"DATE_SOLD",	"NEAREST_SCH_DIST"]].copy()
house_prices_df.head()

SUBURB   PRICE  BEDROOMS  BATHROOMS  GARAGE  LAND_AREA  FLOOR_AREA  \
0  South Lake  565000         4          2     2.0        600         160   
1       Wandi  365000         3          2     2.0        351         139   
2     Camillo  287000         3          1     1.0        719          86   
3    Bellevue  255000         2          1     2.0        651          59   
4   Lockridge  325000         4          1     2.0        466         131   

   CBD_DIST  NEAREST_STN_DIST  DATE_SOLD  NEAREST_SCH_DIST  
0     18300              1800  09-2018\r          0.828339  
1     26900              4900  02-2019\r          5.524324  
2     22600              1900  06-2015\r          1.649178  
3     17900              3600  07-2018\r          1.571401  
4     11200              2000  11-2016\r          1.514922

In [8]:
house_prices_df.columns= house_prices_df.columns.str.lower()
house_prices_df.head()

suburb   price  bedrooms  bathrooms  garage  land_area  floor_area  \
0  South Lake  565000         4          2     2.0        600         160   
1       Wandi  365000         3          2     2.0        351         139   
2     Camillo  287000         3          1     1.0        719          86   
3    Bellevue  255000         2          1     2.0        651          59   
4   Lockridge  325000         4          1     2.0        466         131   

   cbd_dist  nearest_stn_dist  date_sold  nearest_sch_dist  
0     18300              1800  09-2018\r          0.828339  
1     26900              4900  02-2019\r          5.524324  
2     22600              1900  06-2015\r          1.649178  
3     17900              3600  07-2018\r          1.571401  
4     11200              2000  11-2016\r          1.514922

In [9]:
 insp.get_table_names()

['house_prices', 'solar_metadata_waonly', 'Median Income']

In [ ]:
suburb_output ='Output/suburb_prices.csv'

house_prices_df.to_csv(suburb_output,header=True,index=True)

In [ ]:
house_prices_df.to_sql(name='house_prices', con=engine, if_exists='append', index=False)

In [ ]:
#Part 2
solar_post_codes = solar_df["Small Unit Installation Postcode"]
solar_post_codes.describe

In [ ]:
solar_install_qty_sum = solar_df.iloc[:,[1,3,5,7,9,11,13,15,17]].sum(axis=1)
solar_install_qty_sum

In [ ]:
solar_install_kw_sum = solar_df.iloc[:,[2,4,6,8,10,12,14,16,18]].sum(axis=1)
solar_install_kw_sum

In [ ]:
solar_metadata_df = pd.DataFrame({"post_code": solar_post_codes,
                            "installation_qty": solar_install_qty_sum,
                            "rated_output": solar_install_kw_sum}) 
solar_metadata_df .head()

In [ ]:
solar_metadata_WAonly_df = solar_metadata_df[(solar_metadata_df["post_code"] >= 6000) &
                            (solar_metadata_df["post_code"] < 7000)]

solar_metadata_WAonly_df

In [ ]:
solar_metadata_WAonly_df = solar_metadata_WAonly_df.sort_values("post_code")
solar_metadata_WAonly_df = solar_metadata_WAonly_df.reset_index(drop=True)
solar_metadata_WAonly_df

In [ ]:
# Push cleaned DataFrame to a new CSV file
solar_metadata_WAonly_df.to_csv("Output/solar_metadata_WAonly_df.csv",
                  encoding="utf-8", index=False, header=True)

In [ ]:
solar_metadata_WAonly_df.to_sql(name='solar_metadata_waonly', con=engine, if_exists='append', index=False)

In [ ]:
#Part 3
list(median_income.columns)

In [ ]:
# Remove superflus columns for median income
median_income = median_income[['POA_CODE_2021',
 'Median_age_persons',
 'Median_tot_prsnl_inc_weekly',
 'Median_tot_fam_inc_weekly',
 'Median_tot_hhd_inc_weekly']]
median_income.head()

In [ ]:
# Load csv with ABS data for income by hourshold composition
household_income = pd.read_csv('Resources/G33_household_income.csv')
household_income.head()

In [ ]:
# Review columns for relevance
list(household_income.columns)

In [ ]:
# Remove superflus columns for household income
household_income = household_income[['POA_CODE_2021',
 'Negative_Nil_income_Tot',
 'HI_1_149_Tot',
 'HI_150_299_Tot',
 'HI_300_399_Tot',
 'HI_400_499_Tot',
 'HI_500_649_Tot',
 'HI_650_799_Tot',
 'HI_800_999_Tot',
 'HI_1000_1249_Tot',
 'HI_1250_1499_Tot',
 'HI_1500_1749_Tot',
 'HI_1750_1999_Tot',
 'HI_2000_2499_Tot',
 'HI_2500_2999_Tot',
 'HI_3000_3499_Tot',
 'HI_3500_3999_Tot',
 'HI_4000_more_Tot',
 'Partial_income_stated_Tot',
 'All_incomes_not_stated_Tot',
 'Tot_Family_households',
 'Tot_Non_family_households']]
household_income.head()

In [ ]:
# Load csv with ABS data for individual income
individual_incomeA = pd.read_csv('Resources/2021Census_G17A_WA_POA.csv')
individual_incomeB = pd.read_csv('Resources/2021Census_G17B_WA_POA.csv')
individual_incomeC = pd.read_csv('Resources/2021Census_G17C_WA_POA.csv')
# Merge the three individual income CSVs

individual_income = pd.merge(individual_incomeA,individual_incomeB,on="POA_CODE_2021")
individual_income = pd.merge(individual_income,individual_incomeC,on="POA_CODE_2021")
individual_income.head()

In [ ]:
# Review columns for relevance
list(individual_income.columns)

In [ ]:
# Remove columns to remove age split & people totals
individual_income = individual_income[['POA_CODE_2021',
 'M_Neg_Nil_income_Tot',
 'M_1_149_Tot',
 'M_150_299_Tot',
 'M_300_399_Tot',
 'M_400_499_Tot',
 'M_500_649_Tot',
 'M_650_799_Tot',
 'M_800_999_Tot',
 'M_1000_1249_Tot',
 'M_1250_1499_Tot',
 'M_1500_1749_Tot',
 'M_1750_1999_Tot',
 'M_2000_2999_Tot',
 'M_3000_3499_Tot',
 'M_3500_more_Tot',
 'M_PI_NS_ns_Tot',
 'F_Neg_Nil_income_Tot',
 'F_1_149_Tot',
 'F_150_299_Tot',
 'F_300_399_Tot',
 'F_400_499_Tot',
 'F_500_649_Tot',
 'F_650_799_Tot',
 'F_800_999_Tot',
 'F_1000_1249_Tot',
 'F_1250_1499_Tot',
 'F_1500_1749_Tot',
 'F_1750_1999_Tot',
 'F_2000_2999_Tot',
 'F_3000_3499_Tot',
 'F_3500_more_Tot',
 'F_PI_NS_ns_Tot']]
individual_income.head()

In [ ]:
# Merge the three CSVs

median_household_income = pd.merge(median_income,household_income,on="POA_CODE_2021")
median_household_income = pd.merge(median_household_income,individual_income,on="POA_CODE_2021")
median_household_income.head()

In [ ]:
# Review columns for relevance
list(median_household_income.columns)

In [ ]:
# Remove POA from Postcode number

median_household_income["POA_CODE_2021"]=median_household_income["POA_CODE_2021"].str[3:]
median_household_income.head()

In [ ]:
median_household_income.columns= median_household_income.columns.str.lower()
median_household_income.head()

In [ ]:
output_csv='Output/income_data.csv'

median_household_income.to_csv(output_csv,header=True,index=True)

In [ ]:
median_household_income.to_sql(name='Median Income', con=engine, if_exists='append', index=False)